# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

Step 0. Import the json file to MongoDB

To import the data from the `establishments.json` file into MongoDB, use the following command in your Terminal:

```bash
mongoimport --db uk_food --collection establishments --file establishments.json

In [8]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [9]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
db_list = mongo.list_database_names()
if "uk_food" in db_list:
    print("uk_food is built")
else:
    print("uk_food is not built")

In [6]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
collection_list = db.list_collection_names()
print(collection_list)

In [ ]:
# review a document in the establishments collection
sample_document = db['establishments'].find_one()
pprint(sample_document)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "146A Plumstead Rd",
    "AddressLine2": "London",
    "AddressLine3": "",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [ ]:
# Insert the new restaurant into the collection
db['establishments'].insert_one(new_restaurant)

In [ ]:
# Check that the new restaurant was inserted
if 'Penang Flavours' in db['establishments'].distinct("BusinessName"):
    print("New restaurant 'Penang Flavours' was successfully inserted.")
else:
    print("Failed to insert new restaurant.")

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# Define the query to find BusinessTypeID for "Restaurant/Cafe/Canteen"
query = {
    "BusinessType": "Restaurant/Cafe/Canteen"
}

# Define the projection to return only BusinessTypeID and BusinessType fields
projection = {
    "_id": 0,  # Exclude the _id field from the result
    "BusinessTypeID": 1,
    "BusinessType": 1
}

# Find the matching document and return the specified fields
result = db['establishments'].find_one(query, projection)

# Print the result
print(result)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen"
business_type_query = {"BusinessType": "Restaurant/Cafe/Canteen"}
business_type_data = db['establishments'].find_one(business_type_query)

if business_type_data:
    # Extract the BusinessTypeID
    business_type_id = business_type_data.get("BusinessTypeID", "")

    # Update the new restaurant with the correct BusinessTypeID
    new_restaurant_query = {"BusinessName": "Penang Flavours"}
    new_restaurant_update = {"$set": {"BusinessTypeID": business_type_id}}
    db['establishments'].update_one(new_restaurant_query, new_restaurant_update)

    # Confirm that the new restaurant was updated
    updated_restaurant = db['establishments'].find_one(new_restaurant_query)
    if updated_restaurant:
        print("New restaurant 'Penang Flavours' was successfully updated with BusinessTypeID.")
    else:
        print("Failed to update new restaurant.")
else:
    print("BusinessTypeID for 'Restaurant/Cafe/Canteen' not found.")


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_query = {"LocalAuthorityName": "Dover"}
dover_count = db['establishments'].count_documents(dover_query)
print(f"Number of documents with LocalAuthorityName 'Dover': {dover_count}")

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
db['establishments'].delete_many(dover_query)
print(f"{dover_count} documents with LocalAuthorityName 'Dover' deleted.")

In [ ]:
# Check if any remaining documents include Dover
remaining_query = {"LocalAuthorityName": "Dover"}
remaining_count = db['establishments'].count_documents(remaining_query)
if remaining_count == 0:
    print("All 'Dover' documents have been successfully deleted.")
else:
    print(f"There are still {remaining_count} documents with LocalAuthorityName 'Dover'.")

In [ ]:
# Check that other documents remain with 'find_one'
other_document = db['establishments'].find_one()
if other_document:
    print("Other documents remain in the collection.")
else:
    print("No other documents found.")

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
from decimal import Decimal
# Define the query to find documents with latitude and longitude as strings
query = {
    "geocode.latitude": {"$type": "string"},
    "geocode.longitude": {"$type": "string"}
}

# Use update_many to change the data type from String to Decimal
update_result = db['establishments'].update_many(
    query,
    {
        "$set": {
            "geocode.latitude": {"$toDouble": "$geocode.latitude"},
            "geocode.longitude": {"$toDouble": "$geocode.longitude"}
        }
    }
)

# Print the number of documents updated
print(f"{update_result.modified_count} documents updated.")


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Use update_many to set non-rating values to Null
db['establishments'].update_many(
    {"RatingValue": {"$in": non_ratings}},
    {"$set": {"RatingValue": None}}
)

# Use update_many to change the data type from String to Integer for RatingValue
db['establishments'].update_many(
    {"RatingValue": {"$type": "string"}},
    {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}
)

In [ ]:
# Check that the coordinates and rating value are now numbers
sample_document = db['establishments'].find_one()
print(sample_document)